In [ ]:
# calculate memory %
memorypct = udf(lambda x: x[0]/x[1], FloatType())
dfeng = dfeng.withColumn('memory_pct', memorypct(struct('memory_in_use_mb','memory_total_mb')))

# filter delts=T
dfeng = dfeng.filter(col('delta')==True)

In [ ]:
# Build Baseline
df1 = df0.groupby('mac','name','re_ix').agg(avg('temp_celsius').alias('avg_temp'), stddev('temp_celsius').alias('std_temp'), \
                                      avg('memory_pct').alias('avg_memory'), stddev('memory_pct').alias('std_memory'))
df1 = df1.withColumnRenamed('mac','mac1') \
        .withColumnRenamed('name','name1') \
        .withColumnRenamed('re_ix','re_ix1')

df0 = df0.join(df1, (df0.mac==df1.mac1)&(df0.name==df1.name1)&(df0.re_ix==df1.re_ix1), 'inner')

upper = udf(lambda x: x[0]+3*x[1], FloatType())
df0 = df0.withColumn('upper_temp', upper(struct('avg_temp', 'std_temp'))) \
            .withColumn('upper_memory', upper(struct('avg_memory', 'std_memory')))

# Smoothing new data: here also use df0 
df0 = df0.select('mac','name','re_ix','when','temp_celsius','upper_temp','upper_memory','memory_pct','cpu_user')
df0.toPandas().to_csv('health.csv')


In [2]:
# read data locally
import pandas as pd
df0 = pd.read_csv('~/Code/data/health.csv')

In [4]:
df0.describe()

,Unnamed: 0,re_ix,when,temp_celsius,upper_temp,upper_memory,memory_pct,cpu_user
count,401268.000000,401268.000000,4.012680e+05,401268.000000,401268.000000,401268.000000,401268.000000,401268.000000
mean,200633.500000,0.171895,1.572006e+15,54.668561,59.346263,0.333586,0.324011,14.105236
std,115836.238245,0.377290,2.441489e+10,14.612737,15.755067,0.183799,0.184562,11.537746
min,0.000000,0.000000,1.571962e+15,27.000000,29.028042,0.132577,0.129496,1.000000
25%,100316.750000,0.000000,1.571985e+15,44.000000,46.546750,0.159472,0.149538,5.000000
50%,200633.500000,0.000000,1.572006e+15,56.000000,60.411556,0.389648,0.389648,9.000000
75%,300950.250000,0.000000,1.572027e+15,61.000000,68.102320,0.471505,0.459961,24.000000
max,401267.000000,1.000000,1.572048e+15,83.000000,84.850090,0.630271,0.639648,61.000000


In [38]:
# AD per switch for new data
def AD_func(x0,x1):
    if x0 > x1: return 1
    else: return 0
df0['AD_temp'] = df0.apply(lambda x: AD_func(x['temp_celsius'], x['upper_temp']), axis=1)
df0['AD_memory'] = df0.apply(lambda x: AD_func(x['memory_pct'], x['upper_memory']), axis=1)

# Alarm for new data
def alarm_temp(x):
    if x > 70: return 1
    else: return 0
def alarm_memory(x):
    if x >= 0.9: return 1
    else: return 0
def alarm_user(x):
    if x >= 90: return 1
    else: return 0    
df0['alarm_temp'] = df0.temp_celsius.apply(lambda x: alarm_temp(x))
df0['alarm_memory'] = df0.memory_pct.apply(lambda x: alarm_memory(x))
df0['alarm_user'] = df0.cpu_user.apply(lambda x: alarm_user(x))

In [28]:
df0[df0.AD_memory==1].groupby(['mac','re_ix']).count()

,,Unnamed: 0,name,when,temp_celsius,upper_temp,upper_memory,memory_pct,cpu_user,AD_temp,AD_memory,alarm_temp,alarm_memory,alarm_user
mac,re_ix,,,,,,,,,,,,,
0c8126c7054d,0,396,396,396,396,396,396,396,396,396,396,396,396,396
0c8126c71290,0,156,156,156,156,156,156,156,156,156,156,156,156,156
80acacf2f530,0,156,156,156,156,156,156,156,156,156,156,156,156,156
84b59cc279e0,0,48,48,48,48,48,48,48,48,48,48,48,48,48


In [31]:
df0[df0.AD_temp==1].groupby(['mac','re_ix']).count()

,,Unnamed: 0,name,when,temp_celsius,upper_temp,upper_memory,memory_pct,cpu_user,AD_temp,AD_memory,alarm_temp,alarm_memory,alarm_user
mac,re_ix,,,,,,,,,,,,,
84b59cc279e0,0,1776,1776,1776,1776,1776,1776,1776,1776,1776,1776,1776,1776,1776


In [32]:
df0[df0.alarm_temp==1].groupby(['mac','re_ix']).count()

,,Unnamed: 0,name,when,temp_celsius,upper_temp,upper_memory,memory_pct,cpu_user,AD_temp,AD_memory,alarm_temp,alarm_memory,alarm_user
mac,re_ix,,,,,,,,,,,,,
84c1c1d607e0,1,68976,68976,68976,68976,68976,68976,68976,68976,68976,68976,68976,68976,68976


In [36]:
df0[df0.alarm_memory==1].groupby(['mac','re_ix']).count()

,,Unnamed: 0,name,when,temp_celsius,upper_temp,upper_memory,memory_pct,cpu_user,AD_temp,AD_memory,alarm_temp,alarm_memory,alarm_user
mac,re_ix,,,,,,,,,,,,,


In [39]:
df0[df0.alarm_user==1].groupby(['mac','re_ix']).count()

,,Unnamed: 0,name,when,temp_celsius,upper_temp,upper_memory,memory_pct,cpu_user,AD_temp,AD_memory,alarm_temp,alarm_memory,alarm_user
mac,re_ix,,,,,,,,,,,,,


In [45]:
# Avoid duplicate events: if both alarm and AD, take alarm which is more severe
def override(x0, x1):
    if x1 == 1: return 0
    else: return x0
df0['AD_temp1'] = df0.apply(lambda x: override(x['AD_temp'], x['alarm_temp']), axis=1)
df0['AD_memory1'] = df0.apply(lambda x: override(x['AD_memory'], x['alarm_memory']), axis=1)

In [46]:
df0['detection'] = df0['AD_temp1'] + df0['AD_memory1'] + df0['alarm_temp'] + df0['alarm_user'] + df0['alarm_memory']

In [52]:
df0[df0.detection>0].groupby(['mac','re_ix']).count()

,,Unnamed: 0,name,when,temp_celsius,upper_temp,upper_memory,memory_pct,cpu_user,AD_temp,AD_memory,alarm_temp,alarm_memory,alarm_user,AD_temp1,AD_memory1,detection
mac,re_ix,,,,,,,,,,,,,,,,
0c8126c7054d,0,396,396,396,396,396,396,396,396,396,396,396,396,396,396,396,396
0c8126c71290,0,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156
80acacf2f530,0,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156,156
84b59cc279e0,0,1824,1824,1824,1824,1824,1824,1824,1824,1824,1824,1824,1824,1824,1824,1824,1824
84c1c1d607e0,1,68976,68976,68976,68976,68976,68976,68976,68976,68976,68976,68976,68976,68976,68976,68976,68976
